In [ ]:
import folium
import pandas as pd
import json
import matplotlib as plt
import matplotlib
from branca.element import Figure

# Countries GeoInformation

In [ ]:
p_world_countries = f'countries_med_res.json'

with open(p_world_countries, 'rb') as f:
        world_countries = json.load(f)

country_names = [country["properties"]["name"] for country in world_countries["features"]]
        
len(world_countries["features"])

In [ ]:
country_names = sorted(country_names)
country_data = list(zip(country_names, range(len(country_names))))
country_data = pd.DataFrame(country_data, columns=["name", "value"])
#country_data = country_data[:50]

In [ ]:
country_data

# Load Map Data from Indices

In [ ]:
INDICES = {}

## Safety Indices

In [ ]:
p_saf = "safety_index.csv"
DF = pd.read_csv(p_saf)
DF = DF.drop("Rank", axis=1)
DF


In [ ]:
indices = ["Crime", "Safety"]

In [ ]:
# normalize: Germany == 100
for index in indices:
    Z = DF[DF.Country == "Germany"][index].values
    DF[index] = DF[index] / Z * 100

In [ ]:
# add index information to country data
for country_feature in world_countries["features"]:
    country_name = country_feature["properties"]["name"]
    data = DF[DF["Country"] == country_name]
    if len(data) == 0:
        for index in indices:
            country_feature["properties"][index] = None
    else:
        for index in indices:
            country_feature["properties"][index] = float(data[index].values)

In [ ]:
for index in indices:
    INDICES[index] = {"min": DF[index].min(), "max": DF[index].max()}

# Cost Of Living Indices

In [ ]:
indices = ["Cost of Living", "Rent", "Cost of Living Plus Rent",
           "Groceries", "Restaurant Price", "Local Purchasing Power"]

#throw_out_expensive = ["Bermuda", "Switzerland", "Bahamas", "Barbados", "Iceland", "Norway", "Jersey"]

In [ ]:
p_col = "cost_of_living.csv"
DF = pd.read_csv(p_col)
DF = DF.drop("Rank", axis=1)
#
#for country in throw_out_expensive:
#    COL = COL[COL.Country != country]
#COL

In [ ]:
# normalize: Germany is 100, everywhere
for index in indices:
    Z = DF[DF.Country == "Germany"][index].values
    DF[index] = DF[index] / Z * 100

In [ ]:
for country_feature in world_countries["features"]:
    country_name = country_feature["properties"]["name"]
    data = DF[DF["Country"] == country_name]
    if len(data) == 0:
        for index in indices:
            country_feature["properties"][index] = None
    else:
        for index in indices:
            country_feature["properties"][index] = float(data[index].values)

In [ ]:
for index in indices:
    INDICES[index] = {"min": DF[index].min(), "max": DF[index].max()}

In [ ]:
INDICES

# Draw Map

In [ ]:
def get_color_mapper(vmin, vmax, cmap="seismic", ncolors=10):
    normalizer = plt.colors.Normalize(vmin = vmin, vmax = vmax)
    cmap = plt.cm.get_cmap(cmap, ncolors)    # PiYG
    color_map = plt.cm.ScalarMappable(norm=normalizer, cmap=cmap)
    
    def color_mapper(x):
        c = color_map.to_rgba(x)
        c = plt.colors.to_hex(c)
        return c
    return color_mapper

In [ ]:
def get_layer_function(index, color_mapper):
    def layer_function(x):
        """
            fillcolor: color if area
            color: color of border liens
            weight: size of border lines
        """
        country_name = x["properties"]["name"]
        style_dict = {'fillColor': "gray", 'color': 'gray', "opacity": 0, "weight": 0, 'fillOpacity': 1.0}
        if x["properties"][index] is not None:
            c = x["properties"][index]
            c = color_mapper(c)
            style_dict = {'fillColor': c, 'weight':1.0, "color": c, "opacity": 1.0, "fillOpacity": 0.8}
        return style_dict
    return layer_function

In [ ]:
fig=Figure(width=1000,height=600)
m = folium.Map(zoom_start=2)
fig.add_child(m)
folium.TileLayer('Stamen Terrain').add_to(m)
folium.TileLayer('Stamen Toner').add_to(m)
folium.TileLayer('Stamen Water Color').add_to(m)
folium.TileLayer('cartodbpositron').add_to(m)
folium.TileLayer('cartodbdark_matter').add_to(m)


for idx, index in enumerate(INDICES.keys()):
    print(index)
    cmap_name = 'YlGn' # 'YlGnBu'
    ncolors = 10
    vmin = INDICES[index]["min"]
    vmax = INDICES[index]["max"]
    color_mapper = get_color_mapper(vmin=vmin, vmax=vmax, cmap=cmap_name, ncolors=ncolors)

    folium.GeoJson(world_countries,
                   name = index,
                   style_function = get_layer_function(index, color_mapper),
                   tooltip = folium.GeoJsonTooltip(fields=('name', index), aliases=('name', index), labels=True),
                   zoom_on_click=True,
                   show = idx == 0).add_to(m)
folium.LayerControl().add_to(m)
m

# Add Markers

In [ ]:
data = pd.DataFrame({
   'lon':[-58, 2, 145, 30.32, -4.03, -73.57, 36.82, -38.5],
   'lat':[-34, 49, -38, 59.93, 5.33, 45.52, -1.29, -12.97],
   'name':['Buenos Aires', 'Paris', 'melbourne', 'St Petersbourg', 'Abidjan', 'Montreal', 'Nairobi', 'Salvador'],
   'value':[10, 12, 40, 70, 23, 43, 100, 43]
}, dtype=str)

data


In [ ]:
for i in range(0,len(data)):
    folium.Marker(
        location=[data.iloc[i]['lat'], data.iloc[i]['lon']],
        popup=data.iloc[i]['name'],
    ).add_to(m)

# Show the map again
m
